# Performance Comparision between Panda Dataframe and Spark Dataframe

In this exercise, I am interested to find out the performance gain from using Spark dataframe over Panda dataframe. There is the option to restrict the cpu usage through the docker run command.  The objective is to run this notebook under environment with varies CPU allocation, the expectation is Spark should outperforms Panda when CPU allocation is abundant. The operations considered in this performance comparions includes loading dataset into dataframes, window functions, aggregation, sorting, etc.

In [1]:
## Setup
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext
import pandas as pd
import numpy as np

spark = SparkSession \
    .builder \
    .appName("MySession") \
    .getOrCreate()
        
spark.conf.set("spark.executor.instances", 1)
spark.conf.set("spark.executor.cores", 1)
spark.conf.set("spark.cores.max", 1)

In [2]:
import time 

# Credits to Fahim Sakri 
# Source (https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d)
# An annotation for timing a python function
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ("%r  %2.2f ms" % (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [3]:
@timeit
def spark_read_csv(s):
    rdd_df = spark.read.csv(s, header=False, inferSchema=True)
    rdd_df = rdd_df.withColumnRenamed("_c0", "post_type_id") # type 1 = question, type 2 = answer
    rdd_df = rdd_df.withColumnRenamed("_c1", "id")
    rdd_df = rdd_df.withColumnRenamed("_c2", "acceptedAnswerId")
    rdd_df = rdd_df.withColumnRenamed("_c3", "parentId")
    rdd_df = rdd_df.withColumnRenamed("_c4", "score")
    rdd_df = rdd_df.withColumnRenamed("_c5", "tag")
    return rdd_df

@timeit
def pd_read_csv(s):
    return pd.read_csv(s, names=["post_type_id", "id", "acceptedAnswerId", "parentId", "score", "tag"],
                   dtype={'post_type_id': np.int64, 'score': np.float16})

data_file='/data/epfl-big-data-analysis/stackoverflow.csv'
rdd_df = spark_read_csv(data_file)
df = pd_read_csv(data_file)
print(df.shape[0])

'spark_read_csv'  37750.67 ms
'pd_read_csv'  2517.93 ms
8143801


In [6]:
import pyspark.sql.functions as func
from pyspark.sql.window import Window

### Rolling Median

@timeit
def pd_median():
    return df.score.rolling(5).median()

pd_median = pd_median()

#t0 = time.time()
#window_spec = Window.orderBy(rdd_df['id']).rangeBetween(-2, 2)
#rdd_df = rdd_df.withColumn('score_median', func.mean(rdd_df['score']).over(window_spec))
#rdd_df.select('score_median').collect()
#t1 = time.time()

#t1 - t0

'pd_median'  2426.30 ms


KeyboardInterrupt: 

In [ ]:
rdd_df